In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = 'evgenyi_onegin.txt'

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# Delete Unicode spescial symbol \ufeff
text = text.replace('\ufeff', '')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [4]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [5]:
text = text + text

In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
text_as_int, text[:2000], len(text_as_int), len(text)

(array([ 71, 110, 104, ..., 104, 121,   0]),
 'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                                Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Святой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких дум и простоты;\n                        Но так и быть - рукой пристрастной\n                        Прими собранье пестрых глав,\n                        Полусмешных, полупечальных,\n                        Простонародных, идеальных,\n                        Небрежный плод моих забав,\n                        Бессонниц, легких вдохновений,\n                        Незрелых и увядших лет,\n                        Ума холодных наблюде

In [9]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [13]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [16]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [17]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           33536     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 131)           134275    
                                                                 
Total params: 30592899 (116.70 MB)
Trainable params: 305

In [19]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 131), dtype=float32, numpy=
array([[ 1.3255394e-05,  2.1566248e-06, -1.4848262e-05, ...,
         3.2753778e-06,  1.2306136e-05, -4.2851352e-06],
       [ 2.7282162e-05,  6.6262000e-06, -3.6423706e-05, ...,
        -1.8177878e-05,  3.3436947e-05,  8.2152255e-06],
       [ 2.6252223e-05,  2.1742148e-05, -5.4782715e-05, ...,
        -8.3773783e-05,  5.6816985e-05,  6.1400708e-05],
       ...,
       [-1.4324571e-03,  3.2568444e-03, -4.7963746e-03, ...,
         2.6146235e-04,  1.3506110e-03,  3.2545729e-03],
       [-1.1628888e-03,  3.0355649e-03, -4.4587543e-03, ...,
         3.4023129e-04,  1.1814975e-03,  3.0051989e-03],
       [-8.6545944e-04,  2.8217616e-03, -4.1176705e-03, ...,
         3.8646744e-04,  1.0331520e-03,  2.7683191e-03]], dtype=float32)>

In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '\n                        Все в изумленье. Ленский сам\n                        Не верит собственным г'

Next Char Predictions: 
 'ЬщSkiSYмbХХcсВMйEчЕЮhQщфq:НAP}TкДsшТн"ъДSOR"ВD0Ц,iК"lCюЖnTЬ"р.ьASиT;YBp7)rlк-яуuSoуЖбWhlГн5эТBz9сРQм'


In [22]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.873101


In [23]:
model.compile(optimizer='adam', loss=loss)

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=7*10,
    save_weights_only=True)

In [25]:
EPOCHS = 5
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5


88/88 [==============================] - 10739s 121s/step - loss: 2.1941
Epoch 2/5
88/88 [==============================] - 11078s 126s/step - loss: 1.6410
Epoch 3/5
88/88 [==============================] - 10245s 116s/step - loss: 1.4492
Epoch 4/5
88/88 [==============================] - 11948s 136s/step - loss: 1.3712
Epoch 5/5
88/88 [==============================] - 11538s 131s/step - loss: 1.3345


In [26]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_5'

In [27]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


 embedding_1 (Embedding)     (1, None, 256)            33536     
                                                                 
 lstm_4 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_1 (Dense)             (1, None, 131)            134275    
                                                                 
Total params: 30592899 (116.70 MB)
Trainable params: 30592899 (116.70 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
def generate_text(model, start_string, temperature=1, num_generate=500):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = num_generate

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = temperature

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [35]:
text_ = generate_text(model, start_string=u"Что то там такое написано", temperature=1.3)
print(text_)

Что то там такое написаноLЬE26'Нsщ}T}06oyщDaqVЖqLo8{hpР8И}oCpmдЭF}ipBnЮmTI0n{LzЮWFXE3yЛ2у3Е}u3qЮpЬЬ79y3B3qЮэыqЭф2ЭpQ:}oWЕ6nAE}ЦI1onЕТlO3Q3ЭaDqл}:щEL0К}щEpEVщC}ГЗon3NC}NBE8эшqШtW6КЗфtЦpQКHa3БGr}EEE'iCНoЭQ8Ю}LДЬAAЬ МК6Дp2p8Пi'lEК}7ЬБEэrй,ныо, се 

              И мошо! Абпацугке? Вуматявры
                        Уебука Ассьяхищей 6ндыквом
                        накялотлемям Ора омничоцо
        ежгиве,
        Схе ласытет пюреллы вбевтюйл  (Язонея ито
                        Вой,
        Кочнийло? и ллогно:
            


Далее наиболее удачная температура для генерации - при ней модель соскакивает с одного зацикливания на другое:

In [36]:
text_ = generate_text(model, start_string=u"Что то там такое написано", temperature=1.1)
print(text_)

Что то там такое написано9oE61C}oQ"pCCpCЖщ?0}C}WhqLЬЯEq}pD}o3}}7}z:tЖХо}EщъR0q}Б43Цl}5}CХFХ}iL}3ю122Оn8C7B3}ФЭm3v}Ьq1щЬ7oэ}PLNP}80OC;IЮщDФ'l1aНqХ}}tpъLvC33PРWНW3qE0zЦIWp}}CLQE599?C!RLYAщXХnYQE7{}4pMЖATE}2LC8ЮBTEЬD}z'b1rlPwsERCf3R}Ь(03C8ФSЮ}Ю}15Wфю0yWБЭgE2}qpwЭoqEk1w6a0qWЖo}g}Vnp}ХTHh}ЬШW1ЮYl6Ж10HpWК}yъEuPш}3П}юЦэ}щT8WDd}6uGQoъFogCCp}RpPp}R5CP3}AT}D4H0ЧFpWai3kLWКI}1o}w6WбЮ57,}WЭ}q}YYПnnn33ЛuWъC;3XzI}ъФ385QIvIW}CЬН,3LЛъиY6B}kzEw3vВ}9Ae11n{}E2QpХp9Hp}iУиR}pLVЦ}Ь}0Eq18ЗьЮ}yTm33CПьww!щ4p6eEPrpz2q0ЮT6?sL3GЬmCAT}}}Юpьq11zFoЬ}8


In [37]:
text_ = generate_text(model, start_string=u"Что то там такое написано", temperature=.1)
print(text_)

Что то там такое написано}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}p}}}}}}}}}}}}}}}}}}p}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}p}}}}}}}}}}}}}}}}}}}}}}p}p}}}}}}}}}}}}}}}}}}}}}}}}}Ь}}}p}}p}}}}}}}p}}}}}}}}p}p}}p}}}}}}p}}}}}}}p}ppp}pp}}}}}}pp}}}p}}}}pp}}}}}ppppp}}p}}}}}}}}}}}}}}}}}}}}}}p}}}}}}}}}}}}}p}}}}}}}}}}}}}}}}}}}}}}}}}}p}}}pp}}}}}}}}}}}}}p}}}}}}}p}p}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}p}}}}}p}}}}}}}}}}}}}}}}}}}}}}}}}p}p}}}p}}}


In [33]:
text_ = generate_text(model, start_string=u"Что то там такое написано", temperature=.01)
print(text_)

Прекрасны горы под луной }}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}


In [38]:
len(text_)

525

In [ ]:
# Модель требует большого количестов эпох для генерации удобочитаемых текстов.